In [16]:
import pandas as pd
from geopy.distance import geodesic







In [17]:
nb_df = pd.read_csv('../data/raw/nb.csv')
location_df = pd.read_csv('../data/raw/location.csv')

In [18]:
nb_df.info()
nb_df.describe()
nb_df.head(5)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536020 entries, 0 to 536019
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Trip ID           536020 non-null  int64 
 1   Trip Origin       536020 non-null  object
 2   Trip Destination  536020 non-null  object
 3   Trip Start Time   534369 non-null  object
 4   Trip End Time     536019 non-null  object
dtypes: int64(1), object(4)
memory usage: 20.4+ MB


,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


In [19]:
nb_df.info()
nb_df.describe()
location_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536020 entries, 0 to 536019
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Trip ID           536020 non-null  int64 
 1   Trip Origin       536020 non-null  object
 2   Trip Destination  536020 non-null  object
 3   Trip Start Time   534369 non-null  object
 4   Trip End Time     536019 non-null  object
dtypes: int64(1), object(4)
memory usage: 20.4+ MB


,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at
0,1,392001,243828,accepted,6.602207,3.270465,NaN,NaN
1,2,392001,243588,rejected,6.592097,3.287445,NaN,NaN
2,3,392001,243830,rejected,6.596133,3.281784,NaN,NaN
3,4,392001,243539,rejected,6.596142,3.280526,NaN,NaN
4,5,392001,171653,rejected,6.609232,3.288800,NaN,NaN


In [20]:
all_nan_columns_nb = nb_df.columns[nb_df.isna().all()].tolist()
all_nan_columns_location = location_df.columns[location_df.isna().all()].tolist()

if all_nan_columns_nb:
    print("\nColumns in nb_df with all NaN values:")
    for col in all_nan_columns_nb:
        print(col)
else:
    print("\nNo columns in nb_df have all NaN values.")

if all_nan_columns_location:
    print("\nColumns in location_df with all NaN values:")
    for col in all_nan_columns_location:
        print(col)
else:
    print("\nNo columns in location_df have all NaN values.")



No columns in nb_df have all NaN values.

Columns in location_df with all NaN values:
created_at
updated_at


In [21]:
nb_df_cleaned = nb_df.dropna(axis=1, how='all')
location_df_cleaned = location_df.dropna(axis=1, how='all')

In [22]:
nb_df_cleaned = nb_df_cleaned.dropna()
location_df_cleaned = location_df_cleaned.dropna()

In [ ]:
nb_df_cleaned.info()
location_df_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Index: 534368 entries, 0 to 536019
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Trip ID           534368 non-null  int64 
 1   Trip Origin       534368 non-null  object
 2   Trip Destination  534368 non-null  object
 3   Trip Start Time   534368 non-null  object
 4   Trip End Time     534368 non-null  object
dtypes: int64(1), object(4)
memory usage: 24.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
dtypes: float64(2), int64(3), objec

In [ ]:
nb_df_cleaned.to_csv('../data/processed/nb_cleaned.csv')
location_df_cleaned.to_csv('../data/processed/location_cleaned.csv')

In [ ]:
nb_df_cleaned = pd.read_csv('../data/processed/nb_cleaned.csv')
location_df_cleaned = pd.read_csv('../data/processed/location_cleaned.csv')

In [ ]:
location_df_cleaned = location_df_cleaned.head(50)
accepted_orders = location_df_cleaned[location_df_cleaned['driver_action'] == 'accepted']
unfulfilled_orders = location_df_cleaned[location_df_cleaned['driver_action'] == 'rejected']

In [ ]:
def compute_geodesic_distance(coord1, coord2):
    return geodesic(coord1, coord2).meters

def count_within_radius(df, radius=500):
    count_data = []
    for i, row1 in df.iterrows():
        center_coord = (row1['lat'], row1['lng'])
        count_riders = 0
        count_orders = 0
        riders = set()
        orders = set()
        
        for j, row2 in df.iterrows():
            if i != j:
                coord = (row2['lat'], row2['lng'])
                distance = compute_geodesic_distance(center_coord, coord)
                if distance <= radius:
                    riders.add(row2['driver_id'])
                    orders.add(row2['order_id'])
        
        count_riders = len(riders)
        count_orders = len(orders)
        
        count_data.append({
            'center_order_id': row1['order_id'],
            'center_driver_id': row1['driver_id'],
            'count_riders_within_500m': count_riders,
            'count_orders_within_500m': count_orders
        })
    
    return pd.DataFrame(count_data)

accepted_counts = count_within_radius(accepted_orders)
unfulfilled_counts = count_within_radius(unfulfilled_orders)
